# Making first agent:

# Packages:

In [1]:
!pip install sqlalchemy pyodbc python-dotenv requests


# Loading Environment Variables: LLM API Key, Azure SQL

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
conn_str = os.getenv("AZURE_SQL_CONNECTION_STRING")

groq_api_key, conn_str[:50] + "..."


('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',
 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:...')

# Connecting to Azure SQL:

In [3]:
import urllib
import sqlalchemy as sa
from dotenv import load_dotenv
import os

load_dotenv()

raw_conn_str = os.getenv("AZURE_SQL_CONNECTION_STRING")

# URL encode the connection string
params = urllib.parse.quote_plus(raw_conn_str)

engine = sa.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Test connection
try:
    with engine.connect() as conn:
        result = conn.execute(sa.text("SELECT 1"))
        print("Connected to Azure SQL successfully!")
except Exception as e:
    print("Connection failed:", e)


Connection failed: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Unable to complete login process due to delay in login response (258)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)


# Table creation for Storing AI Agents Logs:
Table only needs to be ceated once that's why error showed when attempted to run second time.

In [ ]:

with engine.begin() as conn:  # begin() ensures autocommit for DDL
    conn.execute(sa.text("""
        CREATE TABLE ai_agent_logs (
            id INT IDENTITY(1,1) PRIMARY KEY,
            user_input NVARCHAR(MAX),
            agent_action NVARCHAR(255),
            llm_response NVARCHAR(MAX),
            timestamp DATETIME DEFAULT GETDATE()
        )
    """))

print("Table created successfully with autocommit!")


ProgrammingError: (pyodbc.ProgrammingError) ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'ai_agent_logs' in the database. (2714) (SQLExecDirectW)")
[SQL: 
        CREATE TABLE ai_agent_logs (
            id INT IDENTITY(1,1) PRIMARY KEY,
            user_input NVARCHAR(MAX),
            agent_action NVARCHAR(255),
            llm_response NVARCHAR(MAX),
            timestamp DATETIME DEFAULT GETDATE()
        )
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

# Confirming database connection:

In [6]:
with engine.connect() as conn:
    db_name = conn.execute(sa.text("SELECT DB_NAME()")).fetchone()
db_name



('ai_agent_db',)

# Checking if Table exists:

In [7]:
with engine.connect() as conn:
    rows = conn.execute(sa.text("""
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME = 'ai_agent_logs'
    """)).fetchall()

rows


[('ai_agent_logs',)]

# Groq API call function:

In [8]:
import requests

def call_groq(prompt):
    url = "https://api.groq.com/openai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {groq_api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "llama-3.1-8b-instant",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2
    }
    
    response = requests.post(url, headers=headers, json=data)
    result = response.json()

    # Handle API errors gracefully
    if "error" in result:
        return f"[Groq API Error] {result['error']['message']}"

    # Handle unexpected response format
    if "choices" not in result:
        return f"[Unexpected API Response] {result}"

    return result["choices"][0]["message"]["content"]


# Testing Grog LLM:

In [9]:
call_groq("Say hello in one sentence and tell me how are you today in simple and concise way?")


"Hello, I'm functioning properly today, ready to assist you."

# Ai Agent Logic:

In [10]:
def classify_task(user_input):
    """
    Very simple rule-based classifier.
    In real systems, this could be another LLM call.
    """
    text = user_input.lower()
    
    if "summarize" in text or "summary" in text:
        return "summarization"
    if "extract" in text or "fields" in text:
        return "extraction"
    if "email" in text or "rewrite" in text:
        return "email_rewrite"
    
    return "general_llm"


# Full Agent Function:

In [11]:
import sqlalchemy as sa

def ai_agent(user_input):
    # Step 1: classify
    action = classify_task(user_input)
    
    # Step 2: build prompt based on action
    if action == "summarization":
        prompt = f"Summarize the following text in 3 bullet points:\n\n{user_input}"
    elif action == "extraction":
        prompt = f"Extract key fields from this text and return them as JSON:\n\n{user_input}"
    elif action == "email_rewrite":
        prompt = f"Rewrite this email to be more professional:\n\n{user_input}"
    else:
        prompt = user_input
    
    # Step 3: call Groq LLM
    llm_output = call_groq(prompt)
    
    # Step 4: store in Azure SQL (with autocommit)
    with engine.begin() as conn:
        conn.execute(sa.text("""
            INSERT INTO ai_agent_logs (user_input, agent_action, llm_response)
            VALUES (:user_input, :agent_action, :llm_response)
        """), {
            "user_input": user_input,
            "agent_action": action,
            "llm_response": llm_output
        })
    
    return {
        "action": action,
        "response": llm_output
    }



# Testing  Agent:

In [12]:
# Test 1 — Summarization
ai_agent("Make a paragraph starting with this text and be concise:" \
" Artificial intelligence is transforming industries by automating tasks...")


{'action': 'general_llm',
 'response': 'Artificial intelligence is transforming industries by automating tasks, enhancing decision-making, and improving efficiency. From healthcare to finance, AI-powered systems are analyzing vast amounts of data to identify patterns and make predictions, leading to breakthroughs in disease diagnosis, personalized medicine, and risk management. Additionally, AI-driven chatbots and virtual assistants are revolutionizing customer service, providing 24/7 support and improving overall user experience. As AI continues to evolve, its applications will expand, driving innovation and growth across various sectors.'}

In [13]:
# Test 2 — Extraction
ai_agent("Extract fields: Name: John Doe, Age: 29, Email: john@example.com")

{'action': 'extraction',
 'response': 'Here are the extracted key fields returned as JSON:\n\n```json\n{\n  "Name": "John Doe",\n  "Age": 29,\n  "Email": "john@example.com"\n}\n```\n\nIf you want to generate this JSON in a programming language, here\'s an example in Python:\n\n```python\nimport json\n\ndata = {\n  "Name": "John Doe",\n  "Age": 29,\n  "Email": "john@example.com"\n}\n\nprint(json.dumps(data, indent=4))\n```\n\nThis will output the same JSON as above, but with indentation for readability.'}

In [14]:
# Test 3 — Email Rewrite
ai_agent("Rewrite this email: hey boss, i cant come today lol")


{'action': 'email_rewrite',
 'response': "Here's a rewritten version of the email:\n\nSubject: Notification of Absence\n\nDear [Supervisor's Name],\n\nI am writing to inform you that I will be unable to come to work today. I apologize for any inconvenience this may cause and will make sure to catch up on any missed work as soon as possible.\n\nThank you for your understanding.\n\nSincerely,\n[Your Name]"}

In [15]:
# Test 4 — General LLM
ai_agent("Explain what an AI agent is in simple terms.")


{'action': 'general_llm',
 'response': "An AI agent is a computer program that can think and act like a living being. It's designed to make decisions, solve problems, and interact with its environment, just like a human would.\n\nThink of an AI agent like a robot that can:\n\n1. **Perceive** its surroundings (like seeing and hearing).\n2. **Understand** what it sees and hears (like recognizing objects and sounds).\n3. **Make decisions** based on what it understands (like choosing what to do next).\n4. **Take actions** to achieve its goals (like moving or speaking).\n\nAI agents can be simple, like a chatbot that answers basic questions, or complex, like a self-driving car that navigates through traffic. They're designed to learn and improve over time, making them more efficient and effective at completing tasks.\n\nThere are different types of AI agents, including:\n\n1. **Rule-based agents**: follow a set of rules to make decisions.\n2. **Machine learning agents**: learn from data and

# Azure SQL Logs:

In [16]:
with engine.connect() as conn:
    rows = conn.execute(sa.text("SELECT TOP 10 * FROM ai_agent_logs ORDER BY id DESC")).fetchall()

rows


[(20, 'Explain what an AI agent is in simple terms.', 'general_llm', "An AI agent is a computer program that can think and act like a living being. It's designed to make decisions, solve problems, and interact with its  ... (881 characters truncated) ... ne rule-based and machine learning approaches.\n\nOverall, AI agents are designed to make life easier, more efficient, and more enjoyable for humans.", datetime.datetime(2026, 1, 6, 14, 22, 57, 430000)),
 (19, 'Rewrite this email: hey boss, i cant come today lol', 'email_rewrite', "Here's a rewritten version of the email:\n\nSubject: Notification of Absence\n\nDear [Supervisor's Name],\n\nI am writing to inform you that I will b ... (66 characters truncated) ... this may cause and will make sure to catch up on any missed work as soon as possible.\n\nThank you for your understanding.\n\nSincerely,\n[Your Name]", datetime.datetime(2026, 1, 6, 14, 22, 55, 480000)),
 (18, 'Extract fields: Name: John Doe, Age: 29, Email: john@example.com', 